# CloudWatch

In [2]:
%%time
import boto3
import pandas as pd
from datetime import datetime, timedelta, timezone
import io
import json
from time import sleep
from threading import Thread

import sagemaker
from sagemaker.session import Session
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker import get_execution_role, session, Session, image_uris
from sagemaker.s3 import S3Downloader, S3Uploader
from sagemaker.processing import ProcessingJob
from sagemaker.serializers import CSVSerializer
from sagemaker.model_monitor import DataCaptureConfig
from sagemaker.huggingface.model import HuggingFaceModel

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
CPU times: user 1.55 s, sys: 281 ms, total: 1.83 s
Wall time: 4.61 s


In [3]:
##S3 prefixes
bucket_name = 'aai-540-final-data-east'
region_name = 'us-east-2'
session = sagemaker.Session(boto3.Session(region_name=region_name))
s3 = session.boto_session.client('s3')

prefix = "sagemaker/AIEmotions-ModelQualityMonitor"
data_capture_prefix = f"{prefix}/datacapture"
s3_capture_upload_path = f"s3://{bucket_name}/{data_capture_prefix}"


ground_truth_upload_path = (
    f"s3://{bucket_name}/{prefix}/ground_truth_data/{datetime.now():%Y-%m-%d-%H-%M-%S}"
)

reports_prefix = f"{prefix}/reports"
s3_report_path = f"s3://{bucket_name}/{reports_prefix}"

##Get the model monitor image
monitor_image_uri = image_uris.retrieve(framework="model-monitor", region=region_name)

print("Image URI:", monitor_image_uri)
print(f"Capture path: {s3_capture_upload_path}")
print(f"Ground truth path: {ground_truth_upload_path}")
print(f"Report path: {s3_report_path}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
Image URI: 777275614652.dkr.ecr.us-east-2.amazonaws.com/sagemaker-model-monitor-analyzer
Capture path: s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/datacapture
Ground truth path: s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-28-18-18-03
Report path: s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/reports


In [4]:
df_test_new = pd.read_csv('data/test.csv')
df_test_new

emotion_categories = {
	"anger": ["anger", "annoyance", "disapproval"],
	"disgust": ["disgust"],
	"fear": ["fear", "nervousness"],
	"happy": ["joy", "amusement", "approval", "gratitude"],
	"optimistic": ["optimism", "relief", "pride", "excitement"],
	"affectionate": [ "love", "caring", "admiration",  "desire"],
	"sadness": ["sadness", "disappointment", "embarrassment", "grief",  "remorse"],
	"surprise": ["surprise", "realization", "confusion", "curiosity"],
	"neutral": ["neutral"]
} 

emotion_index_to_label = {index: label for index, label in enumerate(emotion_categories)}
df_test_new['emotions'] = df_test_new['emotions'].apply(lambda x: emotion_index_to_label[x])
df_test_new

,text,emotions
0,Yeah I tried to apply but with being partially...,sadness
1,"First time, health/environmental reasons. Seco...",neutral
2,Wow. I dig that.,affectionate
3,Thank you! We’re currently shoveling ourselves...,happy
4,This is one of my favorite of [NAME] songs alt...,affectionate
...,...,...
5422,"So, the kicker... her brother played on Xbox s...",neutral
5423,It would be hilarious if it was a pasta place too,happy
5424,"Oh damn, that would have been the only correct...",sadness
5425,"At this point, LSC only exists to give sociali...",anger


In [5]:
#Retrieve Model
role = sagemaker.get_execution_role()
s3 = boto3.client('s3')
model_dir = 'models/tuned_model/'
tar_file = 'model.tar.gz'


model_name = f"AIEmotion-model-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"

model_data = f's3://{bucket_name}/{model_dir}{tar_file}'
tensorflow_version = '2.6.3'
transformers_version='4.17.0'
py_version = 'py38'
huggingface_model = HuggingFaceModel(model_data=model_data,
                                     role=role,
                                     transformers_version=transformers_version,
                                     tensorflow_version=tensorflow_version,
                                     py_version=py_version,
                                     entry_point='inference.py'
                                   )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [6]:
endpoint_name = f"AIEmotion-model-quality-monitor-{datetime.utcnow():%Y-%m-%d-%H%M}"
#endpoint_name = 'emotion-ai-endpoint'
print("EndpointName =", endpoint_name)

data_capture_config = DataCaptureConfig(
    enable_capture=True, sampling_percentage=100, destination_s3_uri=s3_capture_upload_path
)

predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.c5.large',
    endpoint_name=endpoint_name,
    data_capture_config=data_capture_config,
)

EndpointName = AIEmotion-model-quality-monitor-2024-02-28-1818
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
-----!

In [7]:
# Predictions as baseline dataset
baseline_prefix = prefix + "/baselining"
baseline_data_prefix = baseline_prefix + "/data"
baseline_results_prefix = baseline_prefix + "/results"

baseline_data_uri = f"s3://{bucket_name}/{baseline_data_prefix}"
baseline_results_uri = f"s3://{bucket_name}/{baseline_results_prefix}"
print(f"Baseline data uri: {baseline_data_uri}")
print(f"Baseline results uri: {baseline_results_uri}")

Baseline data uri: s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data
Baseline results uri: s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/baselining/results


In [8]:
# Grab a sample of the dataset
df_test_subset = df_test_new.sample(100)
df_test_subset = df_test_subset.reset_index(drop=True)

In [9]:
def calculate_predictions(results):
    # Initialize an empty list to collect dictionaries
    predictions = []
    
    for result in results:
        pred_labels = result['predictions']
        
        # For each result, create a dictionary with the desired structure
        predictions.append({'pred_label': pred_labels})
    
    # Create a DataFrame directly from the list of dictionaries
    df_results = pd.DataFrame(predictions)
    
    return df_results

In [10]:
predictions = df_test_subset.apply(lambda x: predictor.predict({'text': x['text']}), axis = 1)
predictions

0          {'predictions': 'neutral'}
1            {'predictions': 'happy'}
2          {'predictions': 'neutral'}
3          {'predictions': 'neutral'}
4            {'predictions': 'anger'}
                   ...               
95           {'predictions': 'anger'}
96      {'predictions': 'optimistic'}
97         {'predictions': 'neutral'}
98           {'predictions': 'anger'}
99    {'predictions': 'affectionate'}
Length: 100, dtype: object

In [11]:
df_preds = calculate_predictions(predictions)
df_preds

,pred_label
0,neutral
1,happy
2,neutral
3,neutral
4,anger
...,...
95,anger
96,optimistic
97,neutral
98,anger


In [12]:
df_test_subset

,text,emotions
0,Most people buy them new because the risk of t...,neutral
1,We don't lmao. Anything with humans will have ...,happy
2,I know a lot of people around my rural area wh...,neutral
3,"I bet he paid, AND requested this masterpiece",neutral
4,It's racist.,anger
...,...,...
95,"As a man with a vasectomy, I want some of that...",neutral
96,i hope so,optimistic
97,Better order some of that VIRILITY DRANK,neutral
98,They don't want equal rights. They want extra ...,affectionate


In [13]:
# Concat datasets
validate_dataset = pd.concat([df_test_subset, df_preds], ignore_index=True, axis = 1)
validate_dataset = validate_dataset.rename(columns = {0: 'text', 
                                                      1: 'true_label', 
                                                      2: 'pred_label', 
                                                      3: 'probability'})

validate_dataset['pred_label'] = validate_dataset['pred_label'].replace({'sad': 'sadness', 'surprised': 'surprise'})
validate_dataset

,text,true_label,pred_label
0,Most people buy them new because the risk of t...,neutral,neutral
1,We don't lmao. Anything with humans will have ...,happy,happy
2,I know a lot of people around my rural area wh...,neutral,neutral
3,"I bet he paid, AND requested this masterpiece",neutral,neutral
4,It's racist.,anger,anger
...,...,...,...
95,"As a man with a vasectomy, I want some of that...",neutral,anger
96,i hope so,optimistic,optimistic
97,Better order some of that VIRILITY DRANK,neutral,neutral
98,They don't want equal rights. They want extra ...,affectionate,anger


In [14]:
# Export to CSV
validate_dataset_name = 'validate_dataset.csv'
validate_dataset.to_csv(validate_dataset_name, index=False) 

In [15]:
# Upload baseline
baseline_dataset_uri = S3Uploader.upload(f"{validate_dataset_name}", baseline_data_uri)
baseline_dataset_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


's3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

## Setup Baseline

In [16]:
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.model_monitor import EndpointInput
from sagemaker.model_monitor.dataset_format import DatasetFormat

baseline_dataset_uri = f's3://{bucket_name}/sagemaker/AIEmotions-ModelQualityMonitor/baselining/data/validate_dataset.csv'

emotions_base_model_quality_monitor = ModelQualityMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.large',
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    sagemaker_session=session,
)

# Name of the model quality baseline job
baseline_job_name = f"AIEmotions-model-baseline-job-{datetime.utcnow():%Y-%m-%d-%H%M}"


# Execute the baseline suggestion job.
job = emotions_base_model_quality_monitor.suggest_baseline(
    job_name=baseline_job_name,
    baseline_dataset=baseline_dataset_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    problem_type="MulticlassClassification",
    inference_attribute="pred_label",
    ground_truth_attribute="true_label",
)
job.wait(logs=False)

INFO:sagemaker:Creating processing-job with name AIEmotions-model-baseline-job-2024-02-28-1822


...........................................................................!

In [17]:
baseline_job = emotions_base_model_quality_monitor.latest_baselining_job

In [18]:
multiclass_metrics = baseline_job.baseline_statistics().body_dict["multiclass_classification_metrics"]
pd.json_normalize(multiclass_metrics).T

,0
confusion_matrix.surprise.surprise,8
confusion_matrix.surprise.happy,0
confusion_matrix.surprise.optimistic,0
confusion_matrix.surprise.anger,0
confusion_matrix.surprise.neutral,3
...,...
weighted_f0_5_best_constant_classifier.standard_deviation,None
weighted_f1_best_constant_classifier.value,0.1225
weighted_f1_best_constant_classifier.standard_deviation,None
weighted_f2_best_constant_classifier.value,0.184906


In [19]:
pd.DataFrame(baseline_job.suggested_constraints().body_dict["multiclass_classification_constraints"]).T

,threshold,comparison_operator
accuracy,0.68,LessThanThreshold
weighted_recall,0.68,LessThanThreshold
weighted_precision,0.710105,LessThanThreshold
weighted_f0_5,0.695124,LessThanThreshold
weighted_f1,0.682136,LessThanThreshold
weighted_f2,0.678075,LessThanThreshold


## Setup continuous model monitoring

In [20]:
# Generate a small sample from our testing data
synthetic_sample = df_test_new.sample(300)
synthetic_sample = synthetic_sample.reset_index(drop=True)

In [21]:
%%time

from time import sleep
from threading import Thread

def invoke_endpoint(ep_name, dataframe):
    i = 0
    for index, row in dataframe.iterrows():
        payload = {'text': row['text']}
        payload_str = json.dumps(payload)
        response = session.sagemaker_runtime_client.invoke_endpoint(
            EndpointName=ep_name,
            ContentType='application/json',
            Body=payload_str,
            InferenceId=str(i),
        )['Body'].read()
        i += 1
        sleep(1)


def invoke_endpoint_forever(df):
    while True:
        try:
            invoke_endpoint(endpoint_name, df)
        except session.sagemaker_runtime_client.exceptions.ValidationError:
            pass


thread = Thread(target=invoke_endpoint_forever, args=(synthetic_sample,))
thread.start()

CPU times: user 213 µs, sys: 25 µs, total: 238 µs
Wall time: 3.32 ms


In [22]:
s3_client = boto3.Session().client("s3")
result = s3_client.list_objects(Bucket=bucket_name, Prefix=data_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get("Contents")]
print("Found Capture Files:")

Found Capture Files:


In [23]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket_name, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"{\"text\": \"No problemo bro an another thing idk if this will affect you but you can't get your gun license if you get ur card\"}","encoding":"JSON"},"endpointOutput":{"observedContentType":"application/json","mode":"OUTPUT","data":"[\n  \"{\\\"slack_app\\\": {\\\"neutral\\\": 0.5091121792793274, \\\"anger\\\": 0.3192923963069916, \\\"surprised\\\": 0.055898092687129974}, \\\"endpointOutput_predictions\\\": \\\"neutral\\\"}\",\n  \"application/json\"\n]","encoding":"JSON"}},"eventMetadata":{"eventId":"67b7deb6-ecd0-4918-8ef5-d89088465358","inferenceId":"100","inferenceTime":"2024-02-26T07:55:56Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"application/json","mode":"INPUT","data":"{\"text\": \"This whole thing is ridiculous, I love reddit sometimes. ... and I'm sure reddit really loves this ridiculousness too!\"}","encoding":"JSON"},"endpointOutput":{"observ

In [24]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "application/json",
      "mode": "INPUT",
      "data": "{\"text\": \"No problemo bro an another thing idk if this will affect you but you can't get your gun license if you get ur card\"}",
      "encoding": "JSON"
    },
    "endpointOutput": {
      "observedContentType": "application/json",
      "mode": "OUTPUT",
      "data": "[\n  \"{\\\"slack_app\\\": {\\\"neutral\\\": 0.5091121792793274, \\\"anger\\\": 0.3192923963069916, \\\"surprised\\\": 0.055898092687129974}, \\\"endpointOutput_predictions\\\": \\\"neutral\\\"}\",\n  \"application/json\"\n]",
      "encoding": "JSON"
    }
  },
  "eventMetadata": {
    "eventId": "67b7deb6-ecd0-4918-8ef5-d89088465358",
    "inferenceId": "100",
    "inferenceTime": "2024-02-26T07:55:56Z"
  },
  "eventVersion": "0"
}


In [25]:
# Generating Ground truth
def ground_truth_with_id(row_data, inference_id):
    return {
        "groundTruthData": {
            "data": row_data['emotions'],
            "encoding": "CSV",
        },
        "eventMetadata": {
            "eventId": str(inference_id),
        },
        "eventVersion": "0",
    }

def upload_ground_truth(df, upload_time):
    ground_records = [json.dumps(ground_truth_with_id(row, idx)) for idx, row in df.iterrows()]
    data_to_upload = "\n".join(ground_records)
    target_s3_uri = f"{ground_truth_upload_path}/{upload_time:%Y/%m/%d/%H/%M%S}.jsonl"
    print(f"Uploading {len(ground_records)} records to", target_s3_uri)
    S3Uploader.upload_string_as_file_body(data_to_upload, target_s3_uri)

def generate_gt_data_forever(df):
    while True:
        upload_ground_truth(df, datetime.utcnow())
        sleep(60 * 60)
        
gt_thread = Thread(target=generate_gt_data_forever, args=(synthetic_sample,))
gt_thread.start()

Uploading 300 records to s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-28-18-18-03/2024/02/28/18/2834.jsonl
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml


In [26]:
from sagemaker.model_monitor import CronExpressionGenerator
from time import gmtime, strftime
##Monitoring schedule name
model_monitor_schedule_name = (f"AIEmotion-model-monitoring-schedule-{datetime.utcnow():%Y-%m-%d-%H%M}")

endpoint_input=EndpointInput(
        endpoint_name=endpoint_name,
        inference_attribute='predictions',
        destination="/opt/ml/processing/input_data"
    )

emotions_base_model_quality_monitor.create_monitoring_schedule(
    monitor_schedule_name=model_monitor_schedule_name,
    endpoint_input=endpoint_input,
    output_s3_uri=s3_report_path,
    constraints=baseline_job.suggested_constraints(),
    ground_truth_input=ground_truth_upload_path,
    problem_type = 'MulticlassClassification',
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker.model_monitor.model_monitoring:Creating Monitoring Schedule with name: AIEmotion-model-monitoring-schedule-2024-02-28-1828


In [27]:
emotions_base_model_quality_monitor.describe_schedule()

{'MonitoringScheduleArn': 'arn:aws:sagemaker:us-east-2:891377274348:monitoring-schedule/AIEmotion-model-monitoring-schedule-2024-02-28-1828',
 'MonitoringScheduleName': 'AIEmotion-model-monitoring-schedule-2024-02-28-1828',
 'MonitoringScheduleStatus': 'Pending',
 'MonitoringType': 'ModelQuality',
 'CreationTime': datetime.datetime(2024, 2, 28, 18, 28, 35, 72000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 2, 28, 18, 28, 35, 155000, tzinfo=tzlocal()),
 'MonitoringScheduleConfig': {'ScheduleConfig': {'ScheduleExpression': 'cron(0 * ? * * *)'},
  'MonitoringJobDefinitionName': 'model-quality-job-definition-2024-02-28-18-28-34-666',
  'MonitoringType': 'ModelQuality'},
 'EndpointName': 'AIEmotion-model-quality-monitor-2024-02-28-1818',
 'ResponseMetadata': {'RequestId': 'e527cef9-9d2f-4b3a-95f6-ce597223a3ab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e527cef9-9d2f-4b3a-95f6-ce597223a3ab',
   'content-type': 'application/x-amz-json-1.1',
   'conten

In [28]:
executions = emotions_base_model_quality_monitor.list_executions()
executions

No executions found for schedule. monitoring_schedule_name: AIEmotion-model-monitoring-schedule-2024-02-28-1828


[]

In [29]:
# Wait for the first execution of the monitoring_schedule
print("Waiting for first execution", end="")
while True:
    execution = emotions_base_model_quality_monitor.describe_schedule().get(
        "LastMonitoringExecutionSummary"
    )
    if execution:
        break
    print(".", end="", flush=True)
    sleep(10)
print()
print("Execution found!")

Waiting for first execution....................................................................................................................................................................................................................
Execution found!


In [30]:
while not executions:
    executions = emotions_base_model_quality_monitor.list_executions()
    print(".", end="", flush=True)
    sleep(10)
latest_execution = executions[-1]
latest_execution.describe()

.

{'ProcessingInputs': [{'InputName': 'groundtruth_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/ground_truth_data/2024-02-28-18-18-03/2024/02/28/18',
    'LocalPath': '/opt/ml/processing/groundtruth/2024/02/28/18',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}},
  {'InputName': 'endpoint_input_1',
   'AppManaged': False,
   'S3Input': {'S3Uri': 's3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/datacapture/AIEmotion-model-quality-monitor-2024-02-28-1818/AllTraffic/2024/02/28/18',
    'LocalPath': '/opt/ml/processing/input_data/AIEmotion-model-quality-monitor-2024-02-28-1818/AllTraffic/2024/02/28/18',
    'S3DataType': 'S3Prefix',
    'S3InputMode': 'File',
    'S3DataDistributionType': 'FullyReplicated',
    'S3CompressionType': 'None'}}],
 'ProcessingOutputConfig': {'Outputs': [{'OutputName': 

In [31]:
latest_execution = emotions_base_model_quality_monitor.list_executions()[-1]
report_uri = latest_execution.describe()["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
    "S3Uri"
]
print("Report Uri:", report_uri)

Report Uri: s3://aai-540-final-data-east/sagemaker/AIEmotions-ModelQualityMonitor/reports/merge


In [32]:
# pd.options.display.max_colwidth = None
# violations = latest_execution.constraint_violations().body_dict["violations"]
# violations_df = pd.json_normalize(violations)
# violations_df.head(10)

## Analyze Model Quality

In [33]:
# Create CloudWatch client
cw_client = boto3.Session().client("cloudwatch")

namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_dimensions = [
    {"Name": "Endpoint", "Value": endpoint_name},
    {"Name": "MonitoringSchedule", "Value": model_monitor_schedule_name},
]

In [34]:
# List metrics through the pagination interface
paginator = cw_client.get_paginator("list_metrics")

for response in paginator.paginate(Dimensions=cw_dimensions, Namespace=namespace):
    model_quality_metrics = response["Metrics"]
    for metric in model_quality_metrics:
        print(metric["MetricName"])

In [35]:
alarm_name = "MODEL_QUALITY_WEIGHTED_F1_SCORE"
alarm_desc = (
    "Trigger an CloudWatch alarm when the weighted f1 score drifts away from the baseline constraints"
)
model_quality_f1_drift_threshold = (
    0.7  ##Setting this threshold purposefully high to see the alarm quickly.
)
metric_name = "weighted_f1"
namespace = "aws/sagemaker/Endpoints/model-metrics"

cw_client.put_metric_alarm(
    AlarmName=alarm_name,
    AlarmDescription=alarm_desc,
    ActionsEnabled=True,
    MetricName=metric_name,
    Namespace=namespace,
    Statistic="Average",
    Dimensions=[
        {"Name": "Endpoint", "Value": endpoint_name},
        {"Name": "MonitoringSchedule", "Value": model_monitor_schedule_name},
    ],
    Period=600,
    EvaluationPeriods=1,
    DatapointsToAlarm=1,
    Threshold=model_quality_f1_drift_threshold,
    ComparisonOperator="LessThanThreshold",
    TreatMissingData="breaching",
)

{'ResponseMetadata': {'RequestId': '54394c44-697d-430d-a4fa-a043ebc0c597',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '54394c44-697d-430d-a4fa-a043ebc0c597',
   'content-type': 'text/xml',
   'content-length': '214',
   'date': 'Wed, 28 Feb 2024 19:04:23 GMT'},
  'RetryAttempts': 0}}

## Release Resources

In [ ]:
emotions_base_model_quality_monitor.delete_monitoring_schedule()
sleep(60)

predictor.delete_endpoint()
predictor.delete_model()


Deleting Monitoring Schedule with name: AIEmotion-model-monitoring-schedule-2024-02-28-1828


INFO:sagemaker.model_monitor.model_monitoring:Deleting Model Quality Job Definition with name: model-quality-job-definition-2024-02-28-18-28-34-666
